### Brief Description

This notebook was used to download insurance data from the USDA RMA


In [9]:
import pandas as pd
import requests
from io import StringIO, BytesIO
import zipfile
import os
from pathlib import Path
from typing import Optional, List
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from io import BytesIO
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')


In [10]:
## Config

BASE_URL = "https://pubfs-rma.fpac.usda.gov/pub/Web_Data_Files/Summary_of_Business/state_county_crop"

OUTPUT_DIR = Path("C:/Users/Arnold/OneDrive/Desktop/CAPSTONE PROJECT/farming_risk_regions/data/raw/insurance_data")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

INTERIM_DIR = Path("C:/Users/Arnold/OneDrive/Desktop/CAPSTONE PROJECT/farming_risk_regions/data/interim/insurance_data")
INTERIM_DIR.mkdir(parents=True, exist_ok=True)

#https://pubfs-rma.fpac.usda.gov/pub/Web_Data_Files/Summary_of_Business/state_county_crop/sobcov_2024.zip

In [11]:
def download_rma_data(year: int) -> pd.DataFrame:
    """
    Download USDA RMA Summary of Business data for a specific year and return as DataFrame.

    - Uses a requests.Session with retry/backoff.
    - Saves the downloaded ZIP and the raw extracted text for traceability.
    - Attempts to read pipe-delimited text (fallback to comma) and handles encoding errors.
    """

    zip_filename = f"sobcov_{year}.zip"
    download_url = f"{BASE_URL}/{zip_filename}"

    print(f"Downloading RMA data for year {year} from {download_url}")

    # Create a session with retries
    session = requests.Session()
    retries = Retry(total=3, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    adapter = HTTPAdapter(max_retries=retries)
    session.mount('https://', adapter)
    session.mount('http://', adapter)

    try:
        resp = session.get(download_url, timeout=30, stream=True)
        resp.raise_for_status()

        # Save ZIP to disk (streamed)
        zip_path = OUTPUT_DIR/ zip_filename
        with open(zip_path, 'wb') as f:
            for chunk in resp.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        print(f"Saved ZIP to {zip_path}")

        # Open the ZIP
        with zipfile.ZipFile(zip_path, 'r') as z:
            namelist = z.namelist()
            print(f"Files in ZIP: {namelist}")

            # Prefer text/csv files
            data_file = next((n for n in namelist if n.lower().endswith(('.txt', '.csv'))), None)
            if not data_file:
                print(f"No text/csv data file found in {zip_filename}")
                return pd.DataFrame()

            print(f"Processing {data_file}")

            with z.open(data_file) as fh:
                raw_bytes = fh.read()

            # Try common encodings
            for enc in ('utf-8', 'latin1', 'cp1252'):
                try:
                    content = raw_bytes.decode(enc)
                    break
                except Exception:
                    content = None
            if content is None:
                print(f"Unable to decode {data_file} with common encodings")
                return pd.DataFrame()

            # Save extracted raw file for traceability
            raw_out = OUTPUT_DIR / f"rma_{year}.txt"
            raw_out.write_text(content, encoding='utf-8', errors='replace')

            # Attempt to read with pipe delimiter first, then comma
            try:
                df = pd.read_csv(StringIO(content), delimiter='|', low_memory=False)
            except Exception:
                try:
                    df = pd.read_csv(StringIO(content), delimiter=',', low_memory=False)
                except Exception as e:
                    print(f"Failed to parse {data_file}: {e}")
                    return pd.DataFrame()

            print(f"Successfully read {len(df)} records for year {year}")
            print(f"Columns: {list(df.columns)}")
            return df

    except requests.exceptions.RequestException as e:
        print(f"Failed to download {download_url}: {e}")
    except zipfile.BadZipFile as e:
        print(f"Bad ZIP file {zip_path}: {e}")
    except Exception as e:
        print(f"Unexpected error processing {zip_filename}: {e}")

    return pd.DataFrame()

In [12]:
START_YEAR = 2014
END_YEAR = 2025

all_dfs: List[pd.DataFrame] = []
for year in range(START_YEAR, END_YEAR + 1):
    df_year = download_rma_data(year)
    if not df_year.empty:
        df_year['source_year'] = year
        all_dfs.append(df_year)

# Concatenate and save a combined file
if all_dfs:
    combined = pd.concat(all_dfs, ignore_index=True)
    combined_file = INTERIM_DIR / f'rma_sobcov_{START_YEAR}_{END_YEAR}.csv'
    combined.to_csv(combined_file, index=False)
    print(f"Saved combined RMA file with {len(combined)} records to {combined_file}")
else:
    print("No RMA data downloaded for the requested years.")


Saved ZIP to C:\Users\Arnold\OneDrive\Desktop\CAPSTONE PROJECT\farming_risk_regions\data\raw\insurance_data\sobcov_2014.zip
Files in ZIP: ['sobcov14.txt']
Processing sobcov14.txt
Saved ZIP to C:\Users\Arnold\OneDrive\Desktop\CAPSTONE PROJECT\farming_risk_regions\data\raw\insurance_data\sobcov_2014.zip
Files in ZIP: ['sobcov14.txt']
Processing sobcov14.txt
Successfully read 129601 records for year 2014
Columns: ['2014', '02', 'AK', '999', 'All Other Counties            ', '9999', 'All Other Crops               ', '90', 'APH       ', 'C    ', 'RCAT', '.5000', '1', '1.1', '0', '1.2', '0.1', 'Acres                    ', '165', '0.2', '13932', '1387', '1387.1', '0.3', '0.4', '0.5', '0.6', '.00']
Successfully read 129601 records for year 2014
Columns: ['2014', '02', 'AK', '999', 'All Other Counties            ', '9999', 'All Other Crops               ', '90', 'APH       ', 'C    ', 'RCAT', '.5000', '1', '1.1', '0', '1.2', '0.1', 'Acres                    ', '165', '0.2', '13932', '1387', '13